<a href="https://colab.research.google.com/github/HansHenseler/masdav2024/blob/main/Part_4_Exercise_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opensearch and log2timeline

Exercise 4:

Master of Advanced Studies in Digital Forensics & Cyber Investigation

Data Analytics and Visualization for Digital Forensics

(c) Hans Henseler, 2024


## 1 The examples are in the Part 4 notebook. This notebook only contains initializations needed to run the exercises at the end

First install Plaso-tools as we did in exercise 3

In [1]:
# various install steps to install plaso tools and dependencies to get plaso working in colab
# -y option is to skip user interaction
# some packages need to be deinstalled and reinstalled to resolve dependencies
# these steps take app. 3 minutes to complete on a fresh colab instance
!add-apt-repository -y ppa:gift/stable
!apt update
!apt-get update
!apt install plaso-tools

Repository: 'deb https://ppa.launchpadcontent.net/gift/stable/ubuntu/ jammy main'
Description:
Periodic releases, contains periodic releases intended for non-development use
More info: https://launchpad.net/~gift/+archive/ubuntu/stable
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/gift-ubuntu-stable-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/gift-ubuntu-stable-jammy.list
Adding key to /etc/apt/trusted.gpg.d/gift-ubuntu-stable.gpg with fingerprint 3ED1EAECE81894B171D7DA5B5E80511B10C598B8
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:7 https://r2u.stat.

In [2]:
# This notebook was tested with version 20220724 (psort.py assumes opensearch and longer elasticsearch)

!psort.py -V

plaso - psort version 20240308


In [3]:
# check if plaso tools were installed by running psort.py

!psort.py -o list


******************************** Output Modules ********************************
         Name : Description
--------------------------------------------------------------------------------
      dynamic : Dynamic selection of fields for a separated value output
                format.
         json : Saves the events into a JSON format.
    json_line : Saves the events into a JSON line format.
          kml : Saves events with geography data into a KML format.
       l2tcsv : CSV format used by legacy log2timeline, with 17 fixed fields.
       l2ttln : Extended TLN 7 field | delimited output.
         null : Output module that does not output anything.
   opensearch : Saves the events into an OpenSearch database.
opensearch_ts : Saves the events into an OpenSearch database for use with
                Timesketch.
        rawpy : native (or "raw") Python output.
          tln : TLN 5 field | delimited output.
         xlsx : Excel Spreadsheet (XLSX) output
----------------------------

In [4]:
!wget -q https://artifacts.opensearch.org/releases/bundle/opensearch/2.15.0/opensearch-2.15.0-linux-x64.tar.gz
!wget -q https://artifacts.opensearch.org/releases/bundle/opensearch/2.15.0/opensearch-2.15.0-linux-x64.tar.gz.sha512

!tar -zxf opensearch-2.15.0-linux-x64.tar.gz
!shasum -a 512 -c opensearch-2.15.0-linux-x64.tar.gz.sha512

opensearch-2.15.0-linux-x64.tar.gz: OK


In [5]:
# change the owner of the Opensearch filetree from root to daemon because Opensearch cannot run as root.

!sudo chown -R daemon:daemon opensearch-2.15.0/

Run Elasticsearch as a daemon process

In [6]:
# start Opensearch from the commandline as user daemon.

!sudo -H -u daemon sh -c  "export OPENSEARCH_INITIAL_ADMIN_PASSWORD='P?ssw0rd1'; opensearch-2.15.0/opensearch-tar-install.sh 1> /content/opensearch-2.15.0/logs/os.log 2> /content/opensearch-2.15.0/logs/os.err &"

In [7]:
# Sleep for few seconds to let the instance start.
import time
time.sleep(30)

Once the instance has been started, grep for opensearch in the processes list to confirm the availability.

In [8]:
# this should look like this:
# daemon     10773       1 99 18:28 ?        00:00:50 /content/opensearch-2.9.0/jdk/bin/java -Xshare:auto -Dopensearch.networkaddress.cache.ttl=60 -Dopensearch.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/opensearch-6691642717485449239 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Dclk.tck=100 -Djdk.attach.allowAttachSelf=true -Djava.security.policy=/content/opensearch-2.9.0/config/opensearch-performance-analyzer/opensearch_security.policy --add-opens=jdk.attach/sun.tools.attach=ALL-UNNAMED -XX:MaxDirectMemorySize=536870912 -Dopensearch.path.home=/content/opensearch-2.9.0 -Dopensearch.path.conf=/content/opensearch-2.9.0/config -Dopensearch.distribution.type=tar -Dopensearch.bundled_jdk=true -cp /content/opensearch-2.9.0/lib/* org.opensearch.bootstrap.OpenSearch
# root       11243     404  0 18:29 ?        00:00:00 /bin/bash -c ps -ef | grep opensearch
# root       11245   11243  0 18:29 ?        00:00:00 grep opensearch

!ps -ef | grep opensearch

daemon      4882       1 99 19:08 ?        00:00:37 /content/opensearch-2.15.0/jdk/bin/java -Xshare:
root        5208     204  0 19:08 ?        00:00:00 /bin/bash -c ps -ef | grep opensearch
root        5210    5208  0 19:08 ?        00:00:00 grep opensearch


Query the base endpoint to retrieve information about the cluster. It may take a bit longer for OpenSearch to fully initialize.

In [11]:
# This should look like this:
# {
#   "name" : "c6713b380721",
#   "cluster_name" : "opensearch",
#   "cluster_uuid" : "iSeFMOJ-QrSuWtndad89hw",
#   "version" : {
#     "distribution" : "opensearch",
#     "number" : "2.15.0",
#     "build_type" : "tar",
#     "build_hash" : "61dbcd0795c9bfe9b81e5762175414bc38bbcadf",
#     "build_date" : "2024-06-20T03:26:49.193630411Z",
#     "build_snapshot" : false,
#     "lucene_version" : "9.10.0",
#     "minimum_wire_compatibility_version" : "7.10.0",
#     "minimum_index_compatibility_version" : "7.0.0"
#   },
#   "tagline" : "The OpenSearch Project: https://opensearch.org/"
# }

!curl -XGET https://localhost:9200 -u admin:P?ssw0rd1 --insecure

{
  "name" : "89b3f1c6549f",
  "cluster_name" : "opensearch",
  "cluster_uuid" : "PgRoOvFYRqmtenm7-5uGeA",
  "version" : {
    "distribution" : "opensearch",
    "number" : "2.15.0",
    "build_type" : "tar",
    "build_hash" : "61dbcd0795c9bfe9b81e5762175414bc38bbcadf",
    "build_date" : "2024-06-20T03:26:49.193630411Z",
    "build_snapshot" : false,
    "lucene_version" : "9.10.0",
    "minimum_wire_compatibility_version" : "7.10.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "The OpenSearch Project: https://opensearch.org/"
}


In [14]:
# This should look like this:
# health status index                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size
# green  open   .opensearch-observability qPII7-a8RVyfAfTZ3bI1tw   1   0          0            0       208b           208b
# green  open   .plugins-ml-config        aJxU7qDGSwqOHmAZEb5Ovg   1   0          1            0      3.8kb          3.8kb
# green  open   .opendistro_security      k9Va2419QaKiPA1SpJpFqA   1   0         10            0     77.4kb         77.4kb

!curl -XGET "https://localhost:9200/_cat/indices?v" -u 'admin:P?ssw0rd1' --insecure

health status index                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .opensearch-observability qPII7-a8RVyfAfTZ3bI1tw   1   0          0            0       208b           208b
green  open   .plugins-ml-config        aJxU7qDGSwqOHmAZEb5Ovg   1   0          1            0      3.8kb          3.8kb
green  open   .opendistro_security      k9Va2419QaKiPA1SpJpFqA   1   0         10            0     77.4kb         77.4kb


In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
# In part 3 (step 3) we stored the mus2019ctf.plaso file in your drive.
#
plaso_file = 'gdrive/MyDrive/mus2019ctf.plaso'
#
# and check if it's there
#
!ls -l $plaso_file

# This should look something like this:
# -rw------- 1 root root 494424064 Aug 10 21:04 gdrive/MyDrive/mus2019ctf.plaso

-rw------- 1 root root 493842432 Aug  5 12:39 gdrive/MyDrive/mus2019ctf.plaso


In [ ]:
# If it's not there you can create it by repeating the following steps
#
# The complete mus2019ctf.plaso file is 450MB and takes a while. After you have created it
# it makes sense to store it in your gdrive so you can reuse it:
#
# plaso_file = 'gdrive/MyDrive/mus2019ctf.plaso'
#
# if not you need to create it with log2timeline.py using the complete windows_filter.yml filter
#
# add a shortcut in your Google drive to this shared drive https://drive.google.com/drive/folders/1KUlZUl4Sy2JzgbuRW-oHjIGFClY2bl75?usp=sharing
# then mount you google drive in this colab (you need to authorize this colab to access your google drive)
#
#!pip install pyparsing==3.1.0
#disk_image = "/content/gdrive/MyDrive/Images/Windows/MUS-CTF-19-DESKTOP-001.E01"
# filter_windows = "/content/gdrive/MyDrive/Testdata/filter_windows.yml"
# !ls -l  $filter_windows
# !log2timeline.py -f $filter_windows --storage_file mus2019ctf.plaso $disk_image --parsers win7 --status_view none
# !cp mus2019ctf.plaso "/content/gdrive/MyDrive"

In [17]:
# modify the file_size definition before running psort.py
!sudo sed -e '/"file_size": {/,/},/c\"file_size": {\n    "type": "long"\n},' /usr/share/plaso/opensearch.mappings -i
!grep -A 2 file_size /usr/share/plaso/opensearch.mappings

"file_size": {
    "type": "long"
},


Use psort to write events to Elasticsearch that we setup earlier. We can use the elastic output format

In [18]:
# run psort.py. It takes about 11-13 minutes to export all rows from the 472MB plaso file to OpenSearch

#
!psort.py -o opensearch --server localhost --port 9200 --opensearch-user admin --opensearch-password P?ssw0rd1 --opensearch_mappings /usr/share/plaso/opensearch.mappings --use_ssl --ca_certificates_file_path /content/opensearch-2.15.0/config/root-ca.pem --index_name newmus2019ctf $plaso_file --status_view none

2024-08-19 19:12:44,384 [WARNING] (MainProcess) PID:6478 <opensearch_output> Note that specifying your OpenSearch password via --opensearch_password or the environment PLASO_OPENSEARCH_PASSWORD can expose the password to other users on the system.
Processing completed.


In [19]:
# Let's take a look again at the indices in our Elasticsearch instance
# This should look like this:
# health status index                        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
# green  open   .plugins-ml-config           UCML1zUaSdOVW-xt1x7MJA   1   0          1            0      3.9kb          3.9kb
# green  open   .opensearch-observability    WFUYHyPoTESfAK39S_eIgw   1   0          0            0       208b           208b
# yellow open   newmus2019ctf                R32IE-4sRkWPgYAJq1GOaA   1   1     805683            0    403.4mb        403.4mb
# yellow open   security-auditlog-2024.08.07 KVXM_wPhT_uCMZg4o8GPQw   1   1         47            0     99.7kb         99.7kb
# green  open   .opendistro_security         LjbhbvqNSnuQ4eDUM8e6Xw   1   0         10            0     78.2kb         78.2kb
#

!curl -X GET "https://localhost:9200/_cat/indices?v" -u admin:P?ssw0rd1 --insecure

health status index                        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .opensearch-observability    qPII7-a8RVyfAfTZ3bI1tw   1   0          0            0       208b           208b
green  open   .plugins-ml-config           aJxU7qDGSwqOHmAZEb5Ovg   1   0          1            0      3.9kb          3.9kb
yellow open   newmus2019ctf                VzaW0h5ZTXGc-aRNMeN6oQ   1   1     530753            0    392.2mb        392.2mb
yellow open   security-auditlog-2024.08.19 XB3sBPyNTdav1jWi7lkrxA   1   1         41            0    136.6kb        136.6kb
green  open   .opendistro_security         k9Va2419QaKiPA1SpJpFqA   1   0         10            0     78.2kb         78.2kb


In [20]:
# we can also see what fields were mapped in this index by Psort.py
#
!curl -XGET "https://localhost:9200/newmus2019ctf/_mapping?format=json&pretty" -u admin:P?ssw0rd1 --insecure

{
  "newmus2019ctf" : {
    "mappings" : {
      "properties" : {
        "access_count" : {
          "type" : "long"
        },
        "account_rid" : {
          "type" : "long"
        },
        "application" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword"
            }
          }
        },
        "birth_droid_file_identifier" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "birth_droid_volume_identifier" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "build_number" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }

In [21]:
# So far we have been accessing information directly with curl from the Opensearch REST API
# The is also an Opensearch Python API that we can use. See https://opensearch.org/docs/latest/clients/python/
#

from opensearchpy import OpenSearch

host = 'localhost'
port = 9200
auth = ('admin', 'P?ssw0rd1') # For testing only. Don't store credentials in code.
ca_certs_path = '/content/opensearch-2.15.0/config/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': 'localhost', 'port': 9200}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    ca_certs = ca_certs_path
)


In [22]:
# We define a Python function to list results

def print_results(response):
  for num, doc in enumerate(response['hits']['hits']):
    print(num, '-->', doc['_source'])

def print_results_detailed(response):
  for num, doc in enumerate(response['hits']['hits']):
    print('\n---------------------------------------------------------------------------------------------------\nresult numer: ',num)
    for key, val in doc['_source'].items():
      print(key, val)

# And for printing facets

def print_facets(agg_dict):
  sum=0
  for field, val in agg_dict:
      print("facets of field ", field,':')
      for bucket in val['buckets']:
        for key in bucket:
          if key=='key':
            print('\t',bucket[key],end='=')
          else:
            print(bucket[key],end='')
            sum = sum + bucket[key]
        print()
      print("total number of hits for ",field," is ",sum)

def print_hit_stats(response):
  print('hit stats:')
  for key, val in response['hits'].items():
      print(key, val)
  print('\n')


# Exercises

## 1 Use elasticsearch to filter events in between 2019-03-12 and 2019-03-22

In [24]:
# see https://opensearch.org/docs/2.0/opensearch/rest-api/search/ and find track_total_hits to get the total number of hits in stead of the max 10.000

query = '{"query": { "query_string": {"query": "datetime:[2019-03-12 TO 2019-03-22]"  }}}'

response = client.search(index="newmus2019ctf", body=query, size=4,track_total_hits=True)
print_hit_stats(response)
print_results_detailed(response)

hit stats:
total {'value': 368414, 'relation': 'eq'}
max_score 1.0
hits [{'_index': 'newmus2019ctf', '_id': 'aBkUbJEBy5ewOPvQU6Ne', '_score': 1.0, '_source': {'data_type': 'windows:evtx:record', 'computer_name': 'DESKTOP-0QT8017', 'event_identifier': 100, 'event_level': 4, 'event_version': 0, 'message_identifier': 100, 'offset': 0, 'provider_identifier': '{6bba3851-2c7e-4dea-8f54-31e5afd029e3}', 'record_number': 4679, 'recovered': False, 'source_name': 'Microsoft-Windows-Diagnosis-DPS', 'strings': ['{180B3A99-8C39-4F12-B631-2031998EFE45}', '{5AE2C742-1D4A-4568-A41A-73B87D7A808B}', '{00000000-0000-0000-0000-000000000000}', '%windir%\\system32\\radardt.dll', '{45DE1EA9-10BC-4F96-9B21-4B6B83DBF476}'], 'user_sid': 'S-1-5-19', 'xml_string': '<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">\n  <System>\n    <Provider Name="Microsoft-Windows-Diagnosis-DPS" Guid="{6BBA3851-2C7E-4DEA-8F54-31E5AFD029E3}"/>\n    <EventID>100</EventID>\n    <Version>0</Version>\n    <Level>4</

## 2 Write a query that performs an aggregation on source_long and source_short (can you find the right field names?)

In [25]:
query = '{"query": {"match_all": {}}, "aggs": { "source_short": { "terms": { "field": "source_short.keyword"}},  "source_long": { "terms": { "field": "source_long.keyword"}}}}'

response = client.search(index="newmus2019ctf", body=query, size=0)

print_hit_stats(response)
print_facets(response['aggregations'].items())

hit stats:
total {'value': 10000, 'relation': 'gte'}
max_score None
hits []


facets of field  source_long :
	 NTFS USN change=292405
	 Registry Key=282435
	 WinEVTX=222198
	 File stat=3792
	 MSIE WebCache container record=1081
	 WinPrefetch=839
	 AppCompatCache Registry Key=742
	 Service/Driver Configuration Registry Key=588
	 Task Cache Registry Key=537
	 System=255
total number of hits for  source_long  is  804872
facets of field  source_short :
	 FILE=296248
	 REG=284373
	 EVT=222198
	 LOG=1296
	 WEBHIST=1282
	 LNK=98
	 PE=86
	 AMCACHE=61
	 OLECF=41
total number of hits for  source_short  is  1610555


## 3 Combine your date range filter from exercise 1 with facet aggregation in exercise 2

In [27]:
querystring = '{ "query_string": {"query": "datetime:[2019-03-18 TO 2019-03-19]"  }}'
facets = '"aggs": { "source_short": { "terms": { "field": "source_short.keyword"}},  "source_long": { "terms": { "field": "source_long.keyword"}}}'
query = '{"query": %s,%s}' % (querystring,facets)
print_hit_stats(response)
response = client.search(index="newmus2019ctf", body=query, size=0)

print_facets(response['aggregations'].items())

hit stats:
total {'value': 10000, 'relation': 'gte'}
max_score None
hits []


facets of field  source_long :
	 NTFS USN change=43212
	 WinEVTX=12808
	 File stat=412
	 MSIE WebCache container record=302
	 Registry Key=250
	 WinPrefetch=230
	 Task Cache Registry Key=58
	 Background Activity Moderator Registry Key=21
	 MSIE WebCache containers record=17
	 Windows Shortcut=17
total number of hits for  source_long  is  57327
facets of field  source_short :
	 FILE=43633
	 EVT=12808
	 REG=332
	 WEBHIST=331
	 LOG=232
	 LNK=17
	 OLECF=13
total number of hits for  source_short  is  114693


In [28]:
print(query)

{"query": { "query_string": {"query": "datetime:[2019-03-18 TO 2019-03-19]"  }},"aggs": { "source_short": { "terms": { "field": "source_short.keyword"}},  "source_long": { "terms": { "field": "source_long.keyword"}}}}


## 4 ***Advanced*** Use opensearch facet aggregation to create a treemap visualisation of a filtered set of events in the index.

####Step 1
The source_short and source_long look interesting for visualisation let's focus on REG, LOG and FILE and run a query

In [29]:
facets = '"aggs": {   "source_long": { "terms": { "field": "source_long.keyword"}}, "source_short": { "terms": { "field": "source_short.keyword"}}}'
daterange = 'datetime:[2019-03-12 TO 2019-03-22]'
# FILE:
querystring = '{ "query_string": {"query": "%s AND  source_short:(FILE OR LOG OR REG)"  }}' % daterange
query = '{"query": %s,%s}' % (querystring,facets)
response = client.search(index="newmus2019ctf", body=query, size=0,track_total_hits=True)
print_facets(response['aggregations'].items())

facets of field  source_long :
	 NTFS USN change=292405
	 Registry Key=18506
	 File stat=1880
	 WinPrefetch=597
	 Task Cache Registry Key=158
	 Service/Driver Configuration Registry Key=65
	 Background Activity Moderator Registry Key=36
	 File entry shell item=25
	 Setup API Log=18
	 System=17
total number of hits for  source_long  is  313707
facets of field  source_short :
	 FILE=294310
	 REG=18787
	 LOG=632
total number of hits for  source_short  is  627436


Why is this not very helpful for a treemap visualisation?

Answer: The problem is that this breakdown is not very useful for a hierarchical visualisation like treemap or sunburst because we would like to know for each value pair (source_long, source_short) how many documents there are

####Step 2
One approach is to aggregate only accross source_long an run 3 separate queries for source_short equal to REG, LOG and FILE respectively and then combine the results into a single dataframe that we can visualise:

In [30]:

facets = '"aggs": {   "source_long": { "terms": { "field": "source_long.keyword"}}}'
daterange = 'datetime:[2019-03-12 TO 2019-03-22]'
# FILE:
querystring_file = '{ "query_string": {"query": "%s AND  source_short:FILE"  }}' % daterange
query_file = '{"query": %s,%s}' % (querystring_file,facets)
# LOG:
querystring_log = '{ "query_string": {"query": "%s AND  source_short:LOG"  }}' % daterange
query_log = '{"query": %s,%s}' % (querystring_log,facets)
# REG
querystring_reg = '{ "query_string": {"query": "%s AND  source_short:REG"  }}' % daterange
query_reg = '{"query": %s,%s}' % (querystring_reg,facets)


response_reg = client.search(index="newmus2019ctf", body=query_reg, size=0,track_total_hits=True)
response_file = client.search(index="newmus2019ctf", body=query_file, size=0,track_total_hits=True)
response_log = client.search(index="newmus2019ctf", body=query_log, size=0,track_total_hits=True)


print('Facets for REG:')
print_facets(response_reg['aggregations'].items())

print('Facet for FILE:')
print_facets(response_file['aggregations'].items())

print('Facets for LOG:')
print_facets(response_log['aggregations'].items())

Facets for REG:
facets of field  source_long :
	 Registry Key=18506
	 Task Cache Registry Key=158
	 Service/Driver Configuration Registry Key=65
	 Background Activity Moderator Registry Key=36
	 AppCompatCache Registry Key=13
	 Winlogon Registry Key=3
	 USB Registry Key=2
	 Network Connection Registry Key=1
	 Run/Run Once Registry Key=1
	 Shutdown Registry Key=1
total number of hits for  source_long  is  18786
Facet for FILE:
facets of field  source_long :
	 NTFS USN change=292405
	 File stat=1880
	 File entry shell item=25
total number of hits for  source_long  is  294310
Facets for LOG:
facets of field  source_long :
	 WinPrefetch=597
	 Setup API Log=18
	 System=17
total number of hits for  source_long  is  632


In [31]:
# create a variation of the print_facets function that outputs values to a list
# containing source_short, source_long and doc_count
#
def buckets_to_list(agg_dict,source_short,list):
  for field, val in agg_dict:
    for bucket in val['buckets']:
      list.append([source_short,bucket['key'],bucket['doc_count']])
  return list

data1 = []
buckets_to_list(response_file['aggregations'].items(),'FILE',data1)
buckets_to_list(response_reg['aggregations'].items(),'REG',data1)
buckets_to_list(response_log['aggregations'].items(),'LOG',data1)

data1

[['FILE', 'NTFS USN change', 292405],
 ['FILE', 'File stat', 1880],
 ['FILE', 'File entry shell item', 25],
 ['REG', 'Registry Key', 18506],
 ['REG', 'Task Cache Registry Key', 158],
 ['REG', 'Service/Driver Configuration Registry Key', 65],
 ['REG', 'Background Activity Moderator Registry Key', 36],
 ['REG', 'AppCompatCache Registry Key', 13],
 ['REG', 'Winlogon Registry Key', 3],
 ['REG', 'USB Registry Key', 2],
 ['REG', 'Network Connection Registry Key', 1],
 ['REG', 'Run/Run Once Registry Key', 1],
 ['REG', 'Shutdown Registry Key', 1],
 ['LOG', 'WinPrefetch', 597],
 ['LOG', 'Setup API Log', 18],
 ['LOG', 'System', 17]]

In [32]:
import pandas as pd
df1 = pd.DataFrame(data1)
df1

,0,1,2
0,FILE,NTFS USN change,292405
1,FILE,File stat,1880
2,FILE,File entry shell item,25
3,REG,Registry Key,18506
4,REG,Task Cache Registry Key,158
5,REG,Service/Driver Configuration Registry Key,65
6,REG,Background Activity Moderator Registry Key,36
7,REG,AppCompatCache Registry Key,13
8,REG,Winlogon Registry Key,3
9,REG,USB Registry Key,2


In [33]:
# set column names

df1.columns = ['source_short','source_long','count']
df1

,source_short,source_long,count
0,FILE,NTFS USN change,292405
1,FILE,File stat,1880
2,FILE,File entry shell item,25
3,REG,Registry Key,18506
4,REG,Task Cache Registry Key,158
5,REG,Service/Driver Configuration Registry Key,65
6,REG,Background Activity Moderator Registry Key,36
7,REG,AppCompatCache Registry Key,13
8,REG,Winlogon Registry Key,3
9,REG,USB Registry Key,2


In [34]:
!pip install --upgrade plotly

import plotly.express as px

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 34.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [35]:
fig = px.treemap(df1, path=['source_short', 'source_long'],values='count')
fig.show()

This was a bit cumbersome approach. In step 3 we'll improve and show a way how to do this with a single opensearch query

####Step 3
Opensearch can also aggregate accros multiple fields combined. This is called multi_terms aggregation which aggregates accross all (source_long, source_short) value pairs. Note: by default opensearch returns 10 buckets max. We set it to 20 here using the size parameter.

See https://opensearch.org/docs/2.0/opensearch/bucket-agg/ for more information about opensearch and multi_term aggregation.

In [36]:
facets = '"aggs": {   "source_long_short": { "multi_terms": { "terms": [{ "field": "source_long.keyword"},{"field": "source_short.keyword"}], "size":20}}}'
daterange = 'datetime:[2019-03-12 TO 2019-03-22]'
# FILE:
querystring = '{ "query_string": {"query": "%s AND  source_short:(FILE OR LOG OR REG)"  }}' % daterange
query = '{"query": %s,%s}' % (querystring,facets)

response = client.search(index="newmus2019ctf", body=query, size=0,track_total_hits=True)
response

{'took': 981,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 313729, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'source_long_short': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': ['NTFS USN change', 'FILE'],
     'key_as_string': 'NTFS USN change|FILE',
     'doc_count': 292405},
    {'key': ['Registry Key', 'REG'],
     'key_as_string': 'Registry Key|REG',
     'doc_count': 18506},
    {'key': ['File stat', 'FILE'],
     'key_as_string': 'File stat|FILE',
     'doc_count': 1880},
    {'key': ['WinPrefetch', 'LOG'],
     'key_as_string': 'WinPrefetch|LOG',
     'doc_count': 597},
    {'key': ['Task Cache Registry Key', 'REG'],
     'key_as_string': 'Task Cache Registry Key|REG',
     'doc_count': 158},
    {'key': ['Service/Driver Configuration Registry Key', 'REG'],
     'key_as_string': 'Service/Driver Configuration Registry Key|REG',
     'd

In [37]:
# we need to redefine our bucket to list function

def buckets_to_list2(agg_dict):
  list = []
  for field, val in agg_dict:
    for bucket in val['buckets']:
      list.append([bucket['key'][1],bucket['key'][0],bucket['doc_count']])
  return list


data2 = buckets_to_list2(response['aggregations'].items())

In [38]:
data2


[['FILE', 'NTFS USN change', 292405],
 ['REG', 'Registry Key', 18506],
 ['FILE', 'File stat', 1880],
 ['LOG', 'WinPrefetch', 597],
 ['REG', 'Task Cache Registry Key', 158],
 ['REG', 'Service/Driver Configuration Registry Key', 65],
 ['REG', 'Background Activity Moderator Registry Key', 36],
 ['FILE', 'File entry shell item', 25],
 ['LOG', 'Setup API Log', 18],
 ['LOG', 'System', 17],
 ['REG', 'AppCompatCache Registry Key', 13],
 ['REG', 'Winlogon Registry Key', 3],
 ['REG', 'USB Registry Key', 2],
 ['REG', 'Network Connection Registry Key', 1],
 ['REG', 'Run/Run Once Registry Key', 1],
 ['REG', 'Shutdown Registry Key', 1],
 ['REG', 'User Account Information Registry Key', 1]]

In [39]:
df2 = pd.DataFrame(data2)
df2.columns = ['source_short','source_long','count']
df2

,source_short,source_long,count
0,FILE,NTFS USN change,292405
1,REG,Registry Key,18506
2,FILE,File stat,1880
3,LOG,WinPrefetch,597
4,REG,Task Cache Registry Key,158
5,REG,Service/Driver Configuration Registry Key,65
6,REG,Background Activity Moderator Registry Key,36
7,FILE,File entry shell item,25
8,LOG,Setup API Log,18
9,LOG,System,17


In [40]:
fig = px.treemap(df2, path=['source_short', 'source_long'],values='count')
fig.show()

So we have the same result as under step 2 but we only needed one query by using multi_term aggregation.

####Step 4
The exercise asked for aggregation accross 3 fields. So let's add the date_type field and deepen our treemap visualisation

In [41]:
facets = '"aggs": {   "source_long_short": { "multi_terms": { "terms": [{ "field": "source_long.keyword"},{"field": "source_short.keyword"},{ "field": "data_type.keyword"}], "size":200}}}'
daterange = 'datetime:[2019-03-12 TO 2019-03-22]'
# FILE:
querystring = '{ "query_string": {"query": "%s "  }}' % daterange
query = '{"query": %s,%s}' % (querystring,facets)

response3 = client.search(index="newmus2019ctf", body=query, size=0,track_total_hits=True)
response3

{'took': 462,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 368414, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'source_long_short': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': ['NTFS USN change', 'FILE', 'fs:ntfs:usn_change'],
     'key_as_string': 'NTFS USN change|FILE|fs:ntfs:usn_change',
     'doc_count': 292405},
    {'key': ['WinEVTX', 'EVT', 'windows:evtx:record'],
     'key_as_string': 'WinEVTX|EVT|windows:evtx:record',
     'doc_count': 54118},
    {'key': ['Registry Key', 'REG', 'windows:registry:key_value'],
     'key_as_string': 'Registry Key|REG|windows:registry:key_value',
     'doc_count': 18505},
    {'key': ['File stat', 'FILE', 'fs:stat'],
     'key_as_string': 'File stat|FILE|fs:stat',
     'doc_count': 1880},
    {'key': ['WinPrefetch', 'LOG', 'windows:prefetch:execution'],
     'key_as_string': 'WinPrefetch|LOG|windows:pr

In [42]:
def buckets_to_list3(agg_dict):
  list = []
  for field, val in agg_dict:
    for bucket in val['buckets']:
      list.append([bucket['key'][2],bucket['key'][1],bucket['key'][0],bucket['doc_count']])
  return list


data3 = buckets_to_list3(response3['aggregations'].items())
data3

[['fs:ntfs:usn_change', 'FILE', 'NTFS USN change', 292405],
 ['windows:evtx:record', 'EVT', 'WinEVTX', 54118],
 ['windows:registry:key_value', 'REG', 'Registry Key', 18505],
 ['fs:stat', 'FILE', 'File stat', 1880],
 ['windows:prefetch:execution', 'LOG', 'WinPrefetch', 597],
 ['msie:webcache:container', 'WEBHIST', 'MSIE WebCache container record', 435],
 ['task_scheduler:task_cache:entry', 'REG', 'Task Cache Registry Key', 158],
 ['windows:registry:service',
  'REG',
  'Service/Driver Configuration Registry Key',
  65],
 ['msie:webcache:containers',
  'WEBHIST',
  'MSIE WebCache containers record',
  37],
 ['windows:lnk:link', 'LNK', 'Windows Shortcut', 37],
 ['windows:registry:bam',
  'REG',
  'Background Activity Moderator Registry Key',
  36],
 ['windows:shell_item:file_entry', 'FILE', 'File entry shell item', 25],
 ['msie:webcache:partitions',
  'WEBHIST',
  'MSIE WebCache partitions record',
  18],
 ['setupapi:log:line', 'LOG', 'Setup API Log', 18],
 ['msie:webcache:cookie', 'WEBHI

In [43]:
df3 = pd.DataFrame(data3)
df3.columns = ['data_type','source_short','source_long','count']
df3

,data_type,source_short,source_long,count
0,fs:ntfs:usn_change,FILE,NTFS USN change,292405
1,windows:evtx:record,EVT,WinEVTX,54118
2,windows:registry:key_value,REG,Registry Key,18505
3,fs:stat,FILE,File stat,1880
4,windows:prefetch:execution,LOG,WinPrefetch,597
5,msie:webcache:container,WEBHIST,MSIE WebCache container record,435
6,task_scheduler:task_cache:entry,REG,Task Cache Registry Key,158
7,windows:registry:service,REG,Service/Driver Configuration Registry Key,65
8,msie:webcache:containers,WEBHIST,MSIE WebCache containers record,37
9,windows:lnk:link,LNK,Windows Shortcut,37


In [44]:
fig = px.treemap(df3, path=['data_type','source_short', 'source_long'],values='count')
fig.show()